In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [2]:
import ee
import geemap
service_account = ' boba-account@boba-430314.iam.gserviceaccount.com '
credentials = ee.ServiceAccountCredentials(service_account,'C:/Users/arj26323/OneDrive - University of Georgia/Documents/GEE AUTH/boba-430314-a19be859bfca.json')

ee.Initialize(credentials)

print(geemap.__version__)

0.34.1


In [4]:
stations_fc = 'C:/Users/arj26323/Documents/Data/Phenology/Weather_stations/stations_ga.csv'
fc_stations = geemap.csv_to_ee(stations_fc, latitude = "Latitude", longitude = "Longitude")

In [5]:
##Pixel extraction functions - Daymet

# def rasterExtraction(image):
#     feature = image.sampleRegions(
#         collection = fc_stations,
#         scale = 1000,
#         tileScale = 8 
#     ).map(lambda f: f.set('date', image.date().format('YYYY-MM-dd')))
#     return feature

In [6]:
##Load Daymet collection
daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4').filterBounds(fc_stations).filterdate


In [7]:
#Daily:
# years_dm = range(2013, 2025)

# daymet_list = daymet.toList(daymet.size())

# result_list = []

# for i in range(daymet.size().getInfo()):  
#     daily_image = ee.Image(daymet_list.get(i))
#     extracted_data = rasterExtraction(daily_image)
    
#     # Convert to dictionary
#     fc_dict = extracted_data.getInfo()
    
#     # Extract feature properties and store in DataFrame
#     features = [feature['properties'] for feature in fc_dict['features']]
#     df = pd.DataFrame(features)
#     result_list.append(df)
    
# station_extract = pd.concat(result_list, ignore_index=True)

# station_extract['date'] = pd.to_datetime(station_extract['date'])

# print(station_extract.head())

In [12]:
def rasterExtraction(image):
    return image.sampleRegions(
        collection=fc_stations,
        scale=1000,
        tileScale=8  # Reducing from 16 to 8 to improve efficiency
    ).map(lambda f: f.set('date', image.date().format('YYYY-MM-dd')))


out_dir = os.path.expanduser('~/Downloads')

# Process in 5-year batches
years = list(range(2013, 2025, 1))

for start_year in years:
    end_year = start_year
    print(f"Processing {start_year} to {end_year}...")

    # Filter data for the 5-year range
    daymet_subset = ee.ImageCollection('NASA/ORNL/DAYMET_V4').filterDate(f"{start_year}-01-01", f"{end_year}-12-31")

    extracted_features = daymet_subset.map(rasterExtraction).flatten()
    fc_dict = extracted_features.getInfo()

    features = [feature['properties'] for feature in fc_dict['features']]
    df = pd.DataFrame(features)
    df['date'] = pd.to_datetime(df['date'])

    # Save file in the specified directory
    filename = os.path.join(out_dir, f"daymet_{start_year}_{end_year}.csv")
    df.to_csv(filename, index=False)
    print(f"Saved {filename}")

print("All batches processed!")

Processing 2013 to 2013...
Saved C:\Users\arj26323/Downloads\daymet_2013_2013.csv
Processing 2014 to 2014...
Saved C:\Users\arj26323/Downloads\daymet_2014_2014.csv
Processing 2015 to 2015...
Saved C:\Users\arj26323/Downloads\daymet_2015_2015.csv
Processing 2016 to 2016...
Saved C:\Users\arj26323/Downloads\daymet_2016_2016.csv
Processing 2017 to 2017...
Saved C:\Users\arj26323/Downloads\daymet_2017_2017.csv
Processing 2018 to 2018...
Saved C:\Users\arj26323/Downloads\daymet_2018_2018.csv
Processing 2019 to 2019...
Saved C:\Users\arj26323/Downloads\daymet_2019_2019.csv
Processing 2020 to 2020...
Saved C:\Users\arj26323/Downloads\daymet_2020_2020.csv
Processing 2021 to 2021...
Saved C:\Users\arj26323/Downloads\daymet_2021_2021.csv
Processing 2022 to 2022...
Saved C:\Users\arj26323/Downloads\daymet_2022_2022.csv
Processing 2023 to 2023...
Saved C:\Users\arj26323/Downloads\daymet_2023_2023.csv
Processing 2024 to 2024...


KeyError: 'date'

In [ ]:
#EXPORT
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'daymet_stationdata_2013.csv')
# station_extract.to_csv(out_csv, index = False)

In [ ]:
#Fin